In [0]:
import numpy as np
import codecs
import regex as re
import spacy
import en_core_web_sm
from collections import defaultdict
from pylab import random
import pandas as pd
from scipy.stats.stats import pearsonr
import word2vec

In [3]:
!pip install word2vec

     |████████████████████████████████| 51kB 1.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for word2vec: filename=word2vec-0.11.1-cp36-none-any.whl size=156401 sha256=5c3fb9bb8b748b1d32a98d9996c7ac8e76d5230dbb4917b018ce4abd1c849201
  Stored in directory: /root/.cache/pip/wheels/f3/7c/ac/fcb6d867f806021c3730fd848970db988b1d0030b5d20c0e02
Successfully built word2vec


 N - количество документов
 M - длина словаря слов
 К - кол-во тематик

In [0]:
def stepE():
    for i in range(N):
        for j in range(M):
            denominator = 0
            for k in range(K):
                p[i, j, k] = fi[j, k] * theta[k, i]
                denominator += p[i, j, k]
            if denominator == 0:
                for k in range(0, K):
                    p[i, j, k] = 0
            else:
                for k in range(0, K):
                    p[i, j, k] /= denominator

In [0]:
def stepM():
    # update fi
    for k in range(0, K):
        denominator = 0
        for j in range(0, M):
            fi[j, k] = 0
            for i in range(0, N):
                fi[j, k] += X[i, j] * p[i, j, k]
            denominator += fi[j, k]
        if denominator == 0:
            for j in range(0, M):
                fi[j, k] = 1.0 / M
        else:
            for j in range(0, M):
                fi[j, k] /= denominator
    # update theta
    for i in range(0, N):
        for k in range(0, K):
            theta[k,i] = 0
            denominator = 0
            for j in range(0, M):
                theta[k,i] += X[i, j] * p[i, j, k]
                denominator += X[i,j];
            if denominator == 0:
                theta[k,i] = 1.0 / K
            else:
                theta[k,i] /= denominator

In [0]:
file = codecs.open('datasets2.txt', 'r', 'utf-8')
documents = [document.strip() for document in file] 
file.close()

In [0]:
file = codecs.open('sw.txt', 'r', 'utf-8')
stopwords = [line.strip() for line in file] 
file.close()

In [0]:
pathCV = 'example3/' #Computer vision articles
texts = []
#получаем каждый документ и создаем корпус
for i in range(10):
    fileObj = codecs.open( str(i)+'.txt', "r", "utf_8_sig" )
    text = fileObj.read() # или читайте по строке
    texts.append(text)
    fileObj.close()

In [0]:
def remove_punctuation(text):
    return re.sub(r'\p{P}+', "", text)
def change_num(text):
    return re.sub(r'[\d]+', "1", text)  

In [0]:
nlp = en_core_web_sm.load()
def lemmatization(text):
    tx = change_num(remove_punctuation(text.lower()))
    doc = nlp(tx)
    return [token.lemma_ for token in doc if token.lemma_ not in stopwords]

In [0]:
X_lem = [lemmatization(text) for text in texts]

In [12]:
N = len(X_lem)
N

10

In [13]:
tokens = defaultdict(int)#
count_words =[]
for doc in X_lem:
    count_word = defaultdict(int)
    for word in doc:
        tokens[word] += 1
        count_word[word] = +1
    count_words.append(count_word)
M = len(tokens)
M

3761

In [0]:
words = []
for key in tokens.keys():
    words.append(key)

In [0]:
X = np.zeros([N, M])

In [0]:
for j,word in enumerate(words):
    for i,doc in enumerate(X_lem):
        X[i,j] = count_words[i][word]

In [17]:
X.shape

(10, 3761)

In [0]:
K = 20    # number of topic
maxIteration = 20
threshold = 10.0
topicWordsNum = 10

In [0]:

fi = random([M, K])
theta = random([K, N])
p = np.zeros([N, M, K])


In [0]:
def initializeParameters():
    for i in range(0, N):
        normalization = sum(theta[:, i])
        for j in range(0, K):
            theta[j, i] /= normalization

    for i in range(0, K):
        normalization = sum(fi[:, i])
        for j in range(0, M):
            fi[j, i] /= normalization

In [0]:
def LogLikelihood():
    loglikelihood = 0
    for i in range(0, N):
        for j in range(0, M):
            tmp = 0
            for k in range(0, K):
                tmp += fi[j, k] * theta[k, i]
            if tmp > 0:
                loglikelihood += X[i, j] * np.log(tmp)
    return loglikelihood

In [22]:
fi

array([[0.87176673, 0.31729223, 0.33423961, ..., 0.73776586, 0.5821576 ,
        0.10646816],
       [0.57982809, 0.64899417, 0.24651361, ..., 0.86113476, 0.20775915,
        0.65628451],
       [0.58236969, 0.74701537, 0.47581509, ..., 0.40210217, 0.6812935 ,
        0.43453442],
       ...,
       [0.36189699, 0.46753938, 0.08807306, ..., 0.61310563, 0.30635311,
        0.67790166],
       [0.87398279, 0.05679042, 0.06743001, ..., 0.9458455 , 0.71524665,
        0.80332285],
       [0.05486812, 0.00372845, 0.26326908, ..., 0.92848622, 0.1447642 ,
        0.90879209]])

In [0]:
initializeParameters()

In [24]:
fi

array([[4.59103391e-04, 1.69984129e-04, 1.77326153e-04, ...,
        3.92181223e-04, 3.15205526e-04, 5.67755232e-05],
       [3.05358112e-04, 3.47688032e-04, 1.30784351e-04, ...,
        4.57761604e-04, 1.12489869e-04, 3.49972192e-04],
       [3.06696610e-04, 4.00201290e-04, 2.52437046e-04, ...,
        2.13749280e-04, 3.68882028e-04, 2.31721096e-04],
       ...,
       [1.90587835e-04, 2.50476588e-04, 4.67259286e-05, ...,
        3.25914399e-04, 1.65872941e-04, 3.61499819e-04],
       [4.60270449e-04, 3.04245428e-05, 3.57740509e-05, ...,
        5.02792096e-04, 3.87265745e-04, 4.28382291e-04],
       [2.88955052e-05, 1.99745466e-06, 1.39673731e-04, ...,
        4.93564261e-04, 7.83816531e-05, 4.84625124e-04]])

In [0]:
import time

In [26]:
oldLoglikelihood = 1
newLoglikelihood = 1
for i in range(0, maxIteration):
    stepE()
    stepM()
    newLoglikelihood = LogLikelihood()
    print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
    if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
        break
    oldLoglikelihood = newLoglikelihood

[ 2020-06-13 18:21:10 ]  1  iteration   -45290.5407468616
[ 2020-06-13 18:21:13 ]  2  iteration   -44416.928322569634
[ 2020-06-13 18:21:16 ]  3  iteration   -43131.711587324164
[ 2020-06-13 18:21:19 ]  4  iteration   -41576.83476504681
[ 2020-06-13 18:21:23 ]  5  iteration   -40156.54618190933
[ 2020-06-13 18:21:26 ]  6  iteration   -39087.14835700173
[ 2020-06-13 18:21:29 ]  7  iteration   -38367.32969396076
[ 2020-06-13 18:21:33 ]  8  iteration   -37947.52763101933
[ 2020-06-13 18:21:36 ]  9  iteration   -37724.39840346361
[ 2020-06-13 18:21:39 ]  10  iteration   -37611.69581141993
[ 2020-06-13 18:21:42 ]  11  iteration   -37549.074362152416
[ 2020-06-13 18:21:46 ]  12  iteration   -37505.58292039872
[ 2020-06-13 18:21:49 ]  13  iteration   -37469.049938428725
[ 2020-06-13 18:21:52 ]  14  iteration   -37434.32581753068
[ 2020-06-13 18:21:55 ]  15  iteration   -37399.12529108484
[ 2020-06-13 18:21:59 ]  16  iteration   -37363.28269388043
[ 2020-06-13 18:22:02 ]  17  iteration   -3732

In [27]:
fi.shape

(3761, 20)

In [0]:
tops_w =[]
for i in range(0, K):
    topicword = []
    ids = fi[:, i].argsort()
    for j in ids:
        topicword.insert(0, words[j])
    tmp = ''
    for word in topicword[0:min(topicWordsNum, len(topicword))]:
        tmp += word + ' '
    tops_w.append(tmp)

In [29]:
tops_w

['hollow note variety flower feed pollen family wide change represent ',
 'botanist imperfect systematic limb expose subtend stalk fourth petallike diagram ',
 'fertilize adaptation decoration affection buy social compound ultraviolet corn cryogenian ',
 'condition grass size rake shrub meadow system urban mini gallon ',
 'pink morning vine lemon gold cook apple stone bulb strong ',
 'shrub typical surround acre holly approach style soil maturation greenish ',
 'dorsal link adaptive drink cluster image remarkable sinauer carnivorous organic ',
 'comprise prevent frigida production expression tobrb1 render boss leaf spring ',
 'leafy unopene gem complex squash light root chrysanthemum familiar lemonade ',
 'meal send slice mature   glass real cucumber citrus invasive ',
 'stamens extension multiple microsporangia irregular significance actinomorphic content zygomorphic filament ',
 'fragrant strange palm maximum microscopic soldier explain scientific kingdom literally ',
 'stratigraphic

In [0]:
data = pd.read_csv('wordsim_similarity_goldstandard.txt',sep='\t', names=['word1','word2','sim'])

In [0]:
word1 = data['word1'].values
word2 = data['word2'].values
scores = data['sim'].values

In [0]:
indexs = []
for k,w in enumerate(zip(word1,word2)):
    if (w[0] in words) and (w[1] in words):
        indexs.append(k)

In [33]:
len(indexs)

28

In [0]:
def cos(A,B):
    SumA = 0 
    SumB = 0
    SumAB = 0
    for i in range(len(A)):
        SumAB += A[i]*B[i]
        SumA += A[i]**2
        SumB += B[i]**2
    return SumAB/(np.sqrt(SumA)*np.sqrt(SumB))

def jac(A,B):

    SumA = 0 
    SumB = 0
    for i in range(len(A)):
        SumA += min(A[i],B[i])
        SumB += max(A[i],B[i])
    return SumA/SumB

def KL(A,B):
    Sum = 0
    for i in range (len(A)):
        if (A[i] != 0) and (B[i] != 0):
            
            Sum += A[i] * np.log(abs(A[i]/B[i]))
    return Sum

def JS(A,B):
    AB = [(u[0]+u[1])/2 for u in zip(A,B)]
    return (KL(A,AB)+KL(B,AB))/2

In [89]:
C =[]
cos_dist = []
jac_dist = []
KL_dist = []
JS_dist = []
SC = []
for j in indexs:
    index_w1 = words.index(word1[j])
    index_w2 = words.index(word2[j])
    A = fi[index_w1]
    B = fi[index_w2]
    
    cos_dist.append(cos(A,B))
    jac_dist.append(jac(A,B))
    KL_dist.append(KL(A,B))
    JS_dist.append(JS(A,B))
    SC.append(scores[j])
    
C.append((pearsonr(cos_dist,SC)[0],pearsonr(jac_dist,SC)[0],pearsonr(KL_dist,SC)[0],pearsonr(JS_dist,SC)[0]))
print(C)   

[(0.1388714361329138, 0.06961769064505717, -0.4581396830579761, 0.11475011959413661)]


In [0]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api

In [0]:
model = Word2Vec(X_lem, min_count = 0, workers=cpu_count())

In [90]:

cos_dist = []
jac_dist = []
KL_dist = []
JS_dist = []
SC = []
for j in indexs:
    
    
    A = model[word1[j]]
    B = model[word2[j]]
    
    cos_dist.append(cos(A,B))
    jac_dist.append(jac(A,B))
    KL_dist.append(KL(A,B))
    JS_dist.append(JS(A,B))
    SC.append(scores[j])
    
C.append((pearsonr(cos_dist,SC)[0],pearsonr(jac_dist,SC)[0],pearsonr(KL_dist,SC)[0],pearsonr(JS_dist,SC)[0]))
print(C)   

[(0.1388714361329138, 0.06961769064505717, -0.4581396830579761, 0.11475011959413661), (0.02648273086425529, -0.29082640468983234, -0.16566613479962577, 0.024905529791701495)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [93]:
print('PLSA:',C[0]) 
print('word2vec:',C[1])

PLSA: (0.1388714361329138, 0.06961769064505717, -0.4581396830579761, 0.11475011959413661)
word2vec: (0.02648273086425529, -0.29082640468983234, -0.16566613479962577, 0.024905529791701495)
